In [ ]:
# Cell 1: Load environment variables and prepare FastAPI
from dotenv import load_dotenv
load_dotenv()   # this reads your existing .env in the project root
import os, openai
# Explicitly pull the key from env
openai.api_key = os.getenv("OPENAI_API_KEY")

# Quick check
assert openai.api_key, "OPENAI_API_KEY not found in environment" # load key in .env
print("Loaded key, length:", len(openai.api_key))
import nest_asyncio
nest_asyncio.apply()  # allow uvicorn in-notebook

from fastapi import FastAPI, Request, Response
from icalendar import Calendar, Event
from datetime import datetime
import pytz, uuid

app = FastAPI()
from pydantic import BaseModel, Field
from typing  import Optional



class ICSRequest(BaseModel):
    title:     str       = Field(..., example="Team sync")
    date:      str       = Field(..., example="2025-05-15")   # or date type
    start:     str       = Field(..., example="10:00")
    end:       str       = Field(..., example="11:00")
    tz:        str       = Field(..., example="America/Toronto")
    location:  Optional[str] = Field(None, example="Zoom Meeting")




AssertionError: OPENAI_API_KEY not found in environment

In [ ]:
# Cell 2: root check
@app.get("/")
def read_root():
    return {"message": "AI Scheduler (notebook) running"}

In [ ]:
from fastapi import Response

@app.post(
    "/generate-ics",
    response_description="An ICS calendar file",
    response_class=Response,
    responses={200: {"content": {"text/calendar": {}}}}
)
async def generate_ics(payload: ICSRequest):
    # now `payload` is a validated ICSRequest
    cal = Calendar()
    cal.add("prodid", "-//AIScheduler//EN")
    cal.add("version", "2.0")

    # parse and localize
    tz = pytz.timezone(payload.tz)
    start = tz.localize(datetime.fromisoformat(f"{payload.date}T{payload.start}"))
    end   = tz.localize(datetime.fromisoformat(f"{payload.date}T{payload.end}"))

    event = Event()
    event.add("uid", str(uuid.uuid4()))
    event.add("dtstamp", datetime.utcnow().replace(tzinfo=pytz.UTC))
    event.add("dtstart", start)
    event.add("dtend",   end)
    event.add("summary", payload.title)
    if payload.location:
        event.add("location", payload.location)

    cal.add_component(event)
    ics_bytes = cal.to_ical()

    return Response(
        content=ics_bytes,
        media_type="text/calendar",
        headers={"Content-Disposition": f"attachment; filename={payload.title}.ics"}
    )


In [ ]:
# Cell 4: background server
import threading, uvicorn

def _run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

thread = threading.Thread(target=_run_server, daemon=True)
thread.start()

print("Server started in background on http://127.0.0.1:8000")


Server started in background on http://127.0.0.1:8000


INFO:     Started server process [27336]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [ ]:
import requests

# Define the payload matching your Pydantic model
payload = {
    "title":    "Desk check",
    "date":     "2025-05-17",
    "start":    "15:00",
    "end":      "15:30",
    "tz":       "America/Toronto",
    "location": "Office"
}

# Send POST to your running API
resp = requests.post(
    "http://127.0.0.1:8000/generate-ics",
    json=payload
)

# Check status and write out the file
if resp.status_code == 200:
    with open("desk_check.ics", "wb") as f:
        f.write(resp.content)
    print("ICS file saved as desk_check.ics")
else:
    print("Error", resp.status_code, resp.text)


ICS file saved as desk_check.ics
